# Guardrails: What Are They and How Can You Use NeMo and Guardrails AI To Safeguard LLMs?

Source: 

Hakan Tekgul, **Guardrails: What Are They and How Can You Use NeMo and Guardrails AI To Safeguard LLMs?,** https://arize.com/blog-course/guardrails-what-are-they-and-how-can-you-use-nemo-and-guardrails-ai-to-safeguard-llms/

Aparna Dhinakaran, **Safeguarding LLMs with Guardrails,** https://medium.com/data-science/safeguarding-llms-with-guardrails-4f5d9f57cff2


Framework:

**Guardrails AI**

Package:
https://www.guardrailsai.com/
https://www.guardrailsai.com/docs/how_to_guides/using_llms

Validators:
https://hub.guardrailsai.com/

Github:
https://github.com/guardrails-ai/guardrails

### Generate bug-free SQL code given a natural language description:

In [1]:
# ! guardrails hub install hub://guardrails/valid_sql
# ! guardrails hub install hub://guardrails/competitor_check
# ! guardrails hub install hub://guardrails/toxic_language
# ! guardrails hub install hub://guardrails/profanity_free
# ! pip install sqlvalidator -q

In [2]:
rail_str = """
<rail version="0.1">
<output>
   <string
       name="generated_sql"
       description="Generate SQL for the given natural language instruction."
       format="bug-free-sql"
       on-fail-bug-free-sql="reask" 
   />
</output>

<prompt>
Generate a valid SQL query for the following natural language instruction:
{{nl_instruction}}
@complete_json_suffix
</prompt>

</rail>
"""

In [3]:
prompt = """

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${gr.complete_xml_suffix}
"""

### Create a guard object:

In [4]:
import guardrails as gd
from rich import print
guard = gd.Guard.for_rail_string(rail_str)

### Wrap your LLM API call with the guard wrapper: 

In [5]:
import os
import openai

# Paste your API key here. Remember to not share publicly
OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

raw_llm_response, validated_response, *rest = guard(
    model="gpt-4o-mini",
    messages=[{"role":"user", "content": prompt}],
    prompt_params={
        "nl_instruction": "Select the name of the employee who has the highest salary."
    },
    max_tokens=2048,
    temperature=0,
)


/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai_frameworks/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [6]:
print(guard.history.last.iterations.last.inputs.messages[0]["content"])

Generate a valid SQL query for the following natural language instruction:

Select the name of the employee who has the highest salary.


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
  <string description="Generate SQL for the given natural language instruction." format="bug-free-sql" 
name="generated_sql" required="true"></string>
</output>

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [7]:
print(validated_response)

{'generated_sql': 'SELECT name FROM employees ORDER BY salary DESC LIMIT 1;'}

In [8]:
print(raw_llm_response)

```json
{"generated_sql": "SELECT name FROM employees ORDER BY salary DESC LIMIT 1;"}
```

In [9]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
    │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
    │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
    │ │ │ user │                                                                                              │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Generate a valid SQL query for the following natural language instruction:                   │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Select the name of the employee who has the highest salary.                                  │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Given below is XML that describes the information to extract from this document and the tags │ │ │
    │ │ │      │ to extract it into.                                                                          │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ <output>                                                                                     │ │ │
    │ │ │      │   <string description="Generate SQL for the given natural language instruction."             │ │ │
    │ │ │      │ format="bug-free-sql" name="generated_sql" required="true"></string>                         │ │ │
    │ │ │      │ </output>                                                                                    │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in  │ │ │
    │ │ │      │ JSON is the `name` attribute of the corresponding XML, and the value is of the type          │ │ │
    │ │ │      │ specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including │ │ │
    │ │ │      │ any types and format requests e.g. requests for lists, objects and specific types. Be        │ │ │
    │ │ │      │ correct and concise. If you are unsure anywhere, enter `null`.                               │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:               │ │ │
    │ │ │      │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`          │ │ │
    │ │ │      │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE',        │ │ │
    │ │ │      │ 'STRING TWO', etc.]}`                                                                        │ │ │
    │ │ │      │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer            │ │ │
    │ │ │      │ name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index':     │ │ │
    │ │ │      │ 1}}`                                                                                         │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │                                   

### Create a GuardrailsOutputParser:

In [10]:
from rich import print
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

output_parser = GuardrailsOutputParser.from_rail_string(rail_str, api=openai.ChatCompletion.create)

### Basic example of how to integrate Guardrails with a LangChain LCEL chain:
https://www.guardrailsai.com/docs/integrations/langchain

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

model = ChatOpenAI(model="gpt-4")

In [12]:
from guardrails import Guard

from guardrails_grhub_competitor_check import CompetitorCheck
from guardrails_grhub_valid_sql import ValidSQL
from guardrails_grhub_toxic_language import ToxicLanguage

competitors_list = ["delta", "american airlines", "united"]
guard = Guard().use_many(
    CompetitorCheck(competitors=competitors_list, on_fail="fix"),
    ToxicLanguage(on_fail="filter"),
)

In [13]:
prompt = ChatPromptTemplate.from_template("Answer this question {question}")
output_parser = StrOutputParser()

chain = prompt | model | guard.to_runnable() | output_parser

In [14]:
result = chain.invoke({"question": "What are the top five airlines for domestic travel in the US?"})
print(result)

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai_frameworks/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


1. Southwest Airlines
2. Delta Air Lines
3. American Airlines
4. United Airlines
5. JetBlue Airways

In [18]:
from guardrails import Guard
from guardrails_grhub_profanity_free import ProfanityFree

# Create a Guard class
guard = Guard().use(ProfanityFree())

prompt = ChatPromptTemplate.from_template("Answer this question {question}")
output_parser = StrOutputParser()

chain = prompt | model | guard.to_runnable() | output_parser

In [19]:
result = chain.invoke({"question": "Can you generate a list of 10 things that are not food?"})
print(result)

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai_frameworks/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


1. A pencil
2. A computer
3. A smartphone
4. A chair
5. A car
6. A book
7. A television
8. A pair of shoes
9. A painting
10. A lamp

### Other example usage:

#### OpenAI

In [15]:
import os
from guardrails import Guard

guard = Guard()

OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

result = guard(
    messages=[{"role":"user", "content":"How many moons does Jupiter have?"}],
    model="gpt-4o",
)

print(f"{result.validated_output}")

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai_frameworks/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


As of the latest data, Jupiter has 95 confirmed moons. This number can change as new moons are discovered and 
confirmed by astronomers.

In [16]:
from pydantic import BaseModel, Field
from typing import List
from guardrails import Guard

class Fruit(BaseModel):
    name: str
    color: str

class Basket(BaseModel):
    fruits: List[Fruit]
    
guard = Guard.for_pydantic(Basket)

result = guard(
    messages=[{"role":"user", "content":"Generate a basket of 5 fruits"}],
    model="gpt-4o",
    tools=guard.json_function_calling_tool([]),
    tool_choice="required",
)

print(f"{result.validated_output}")

{'fruits': [{'name': 'Apple', 'color': 'Red'}, {'name': 'Banana', 'color': 'Yellow'}, {'name': 'Grapes', 'color': 
'Purple'}, {'name': 'Orange', 'color': 'Orange'}, {'name': 'Kiwi', 'color': 'Green'}]}

#### Anthropic

In [21]:
# from guardrails import Guard
# import os

# guard = Guard()

# ANTHROPIC_API_KEY = open("/Users/mjack6/.secrets/anthropic_mjack.apikey", "r").read().strip()
# os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# result = guard(
#     messages=[{"role":"user", "content":"How many moons does Jupiter have?"}],
#     model="claude-3-opus-20240229",
#     set_verbose=True,
# )

# print(f"{result.validated_output}")

#### Gemini

In [22]:
# from guardrails import Guard
# import os

# guard = Guard()

# GEMINI_API_KEY = open("/Users/mjack6/.secrets/gemini_mjack.apikey", "r").read().strip()
# os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

# result = guard(
#     messages=[{"role":"user", "content":"How many moons does Jupiter have?"}],
#     model="gemini/gemini-pro"
# )

# print(f"{result.validated_output}")